In [ ]:
! pip install semopy

In [ ]:
import pandas as pd
import numpy as np
gdp = pd.read_csv("GDPC1.csv", parse_dates=[0], index_col=0)
tariffs = pd.read_csv("B235RC1Q027SBEA.csv", parse_dates=[0], index_col=0)
epu = pd.read_csv("USEPUINDXD.csv", parse_dates=[0], index_col=0)
imports = pd.read_csv("IMPGS.csv", parse_dates=[0], index_col=0)

# Merge into a single DataFrame
df = gdp.join([tariffs, epu, imports], how='outer')
df.columns = ['Real_GDP', 'Tariff_Revenue', 'EPU', 'Real_Imports']
df.head()
df.to_excel("merged_data.xlsx")

In [ ]:
df_from_1959 = df.iloc[50:]
df_from_1959.to_excel("df_from_1959.xlsx")
epu_quarterly = epu.resample('QE').mean()
#synthetic integration of epu
epu_quarterly.loc['2017-07-01'] = 110
epu_quarterly.loc['2017-10-01'] = 115
epu_quarterly.loc['2018-01-01'] = 120
epu_quarterly.loc['2018-04-01'] = 130


new_df = pd.concat([gdp, tariffs, epu_quarterly, imports], axis=1)
new_df.columns = ['Real_GDP', 'Tariff_Revenue', 'EPU_QUARTERLY', 'Real_Imports']
(new_df.iloc[50:]).to_excel("semifinal_data.xlsx")



In [ ]:
import pandas as pd
import numpy as np
import semopy
from semopy import Model
data = pd.read_excel("final_data_sem (3).xlsx", index_col=0)
data.head()
data.dropna(inplace=True)
data.to_excel("SEM_MODEL_DATA.xlsx")
#find correlation between all numberic column once
data.corr()

,Real_GDP,Tariff_Revenue,EPU_QUARTERLY,Real_Imports
Real_GDP,1.000000,0.673790,-0.182088,0.935257
Tariff_Revenue,0.673790,1.000000,-0.044421,0.726885
EPU_QUARTERLY,-0.182088,-0.044421,1.000000,-0.241680
Real_Imports,0.935257,0.726885,-0.241680,1.000000


In [ ]:
# Define SEM structure
##model_desc = """
# Structural model
#Real_GDP ~ Tariff_Revenue + EPU_QUARTERLY + Real_Imports
##Real_Imports ~ Tariff_Revenue + EPU_QUARTERLY
##"""

# Fit the SEM model
##model = Model(model_desc)
##results = model.fit(data)

# Inspect parameter estimates
estimates = model.inspect()
print("Parameter Estimates:")
print(estimates)

# Model fit statistics
fit_stats = model.inspect(what='fit')
print("\nModel Fit Statistics:")
print(fit_stats)

#CFI and TLI: should be > 0.90

#RMSEA: ideally < 0.08

#Chi-square and DoF: lower values indicate better fit

Parameter Estimates:
           lval  op            rval  ...       Std. Err   z-value       p-value
0  Real_Imports   ~  Tariff_Revenue  ...       4.063897  4.967382  6.786289e-07
1  Real_Imports   ~   EPU_QUARTERLY  ...       1.042417 -1.452129  1.464657e-01
2      Real_GDP   ~  Tariff_Revenue  ...      14.086754 -0.009365  9.925279e-01
3      Real_GDP   ~   EPU_QUARTERLY  ...       2.646364  0.309228  7.571478e-01
4      Real_GDP   ~    Real_Imports  ...       0.496839  4.643651  3.423053e-06
5  Real_Imports  ~~    Real_Imports  ...   40390.817494  3.464102  5.320055e-04
6      Real_GDP  ~~        Real_GDP  ...  239290.482146  3.464102  5.320055e-04

[7 rows x 7 columns]

Model Fit Statistics:
           lval  op            rval  ...       Std. Err   z-value       p-value
0  Real_Imports   ~  Tariff_Revenue  ...       4.063897  4.967382  6.786289e-07
1  Real_Imports   ~   EPU_QUARTERLY  ...       1.042417 -1.452129  1.464657e-01
2      Real_GDP   ~  Tariff_Revenue  ...      14.08675

In [ ]:
from semopy import Model
import pandas as pd

# Load your dataset
df = pd.read_excel("SEM_MODEL_DATA.xlsx")

# Define SEM model
model_desc = """
Real_GDP ~ Tariff_Revenue + EPU_QUARTERLY + Real_Imports
Real_Imports ~ Tariff_Revenue + EPU_QUARTERLY
"""

# Fit SEM model
model = Model(model_desc)
model.fit(df)
estimates = model.inspect()

# Inspect parameter estimates
estimates = model.inspect()
print("Parameter Estimates:")
print(estimates)

# Model fit statistics
fit_stats = model.inspect(what='fit')
print("\nModel Fit Statistics:")
print(fit_stats)

# Create a new column with full parameter names and set it as index
estimates['param_name'] = estimates['lval'] + estimates['op'] + estimates['rval']
coeffs = estimates.set_index('param_name')['Estimate']

# Extract coefficients
b_tariff = coeffs['Real_GDP~Tariff_Revenue']
b_epu = coeffs['Real_GDP~EPU_QUARTERLY']
b_imports = coeffs['Real_GDP~Real_Imports']
import_tariff = coeffs['Real_Imports~Tariff_Revenue']
import_epu = coeffs['Real_Imports~EPU_QUARTERLY']

print("b_tariff:", b_tariff)
print("b_epu:", b_epu)
print("b_imports:", b_imports)
print("import_tariff:", import_tariff)
print("import_epu:", import_epu)

# Create hypothetical scenarios
def predict_gdp(tariff, epu):
    imports = import_tariff * tariff + import_epu * epu
    gdp = b_tariff * tariff + b_epu * epu + b_imports * imports
    return gdp

# Simulate
gdp_baseline = predict_gdp(90, 125)
gdp_trump2025 = predict_gdp(115, 160)

print("Baseline GDP:", gdp_baseline)
print("Trump 2025 Tariff Scenario GDP:", gdp_trump2025)
print("Estimated GDP Drop:", gdp_baseline - gdp_trump2025)

Parameter Estimates:
           lval  op            rval  ...       Std. Err   z-value       p-value
0  Real_Imports   ~  Tariff_Revenue  ...       4.063897  4.967382  6.786289e-07
1  Real_Imports   ~   EPU_QUARTERLY  ...       1.042417 -1.452129  1.464657e-01
2      Real_GDP   ~  Tariff_Revenue  ...      14.086754 -0.009365  9.925279e-01
3      Real_GDP   ~   EPU_QUARTERLY  ...       2.646364  0.309228  7.571478e-01
4      Real_GDP   ~    Real_Imports  ...       0.496839  4.643651  3.423053e-06
5  Real_Imports  ~~    Real_Imports  ...   40390.817494  3.464102  5.320055e-04
6      Real_GDP  ~~        Real_GDP  ...  239290.482146  3.464102  5.320055e-04

[7 rows x 7 columns]

Model Fit Statistics:
           lval  op            rval  ...       Std. Err   z-value       p-value
0  Real_Imports   ~  Tariff_Revenue  ...       4.063897  4.967382  6.786289e-07
1  Real_Imports   ~   EPU_QUARTERLY  ...       1.042417 -1.452129  1.464657e-01
2      Real_GDP   ~  Tariff_Revenue  ...      14.08675